<a href="https://colab.research.google.com/github/bucuram/machine-translation-labs/blob/main/Lab5_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###IBM Model 1 

Read the [Word Alignment Slides](https://ufal.mff.cuni.cz/~bojar/courses/npfl087/1819/04-word-alignment-Koehn.pdf)  

Watch the [video](http://mttalks.ufal.ms.mff.cuni.cz/index.php?title=Word_Alignment)

The algorithm for IBM-1 looks at lexical translation probability. The goal is to fill in a matrix which says for each target word $w_{t}$ and source word $w_{s}$ what is the probability $P(w_{t}|w_{s})$. Once we have this dictionary, we can get the actual alignments by finding the most probable target word for each source word 

####IBM Model 1 and EM

* Expectation-Step: apply current model parameters to the data
– parts of the model are hidden (here: alignments)

– using the model, assign probabilities to possible values

* Maximization-Step: estimate new model parameters from the data

– take assign values as fact

– collect counts (weighted by probabilities)

– estimate model from counts

* Iterate these steps until convergence


####Example

The corpus

![step1](https://i.imgur.com/qNFg66d.png)

**Maximization step**

The dictionary

![step2](https://i.imgur.com/KH8tB7h.png)

**Expectation Step**

![step3](https://i.imgur.com/lYYMdje.png)

Normalization

![step4](https://i.imgur.com/Y52bjZy.png)

Normalized probabilities

![step5](https://i.imgur.com/ENgYL6m.png)

**Maximization step**

![step6](https://i.imgur.com/aBHhzKc.png)

Finally we have the translation probabilities

![step6](https://i.imgur.com/DZ4aZmM.png)

####Limitations

- It has no notion of **word position** so it is happy to align two neighboring words in the source sentence to two completely different positions in the target.

-  It also disregards how many alignment links lead to a particular word (it does not model the so-called word fertility), so in some cases, it can even align the whole source sentence to a single target-side word, leaving other target words unaligned.

###Assignment

- Apply the IBM model 1 to the romanian - aromanian corpus below

- Extract the dictionary and evaluate the alignments (you may use the [Alignment Error Rate](https://www.nltk.org/howto/align.html))

- Search for 10 examples of words that are misaligned and think about the solutions to overcome this wrong alignments

- Use the IBM model to translate from the source corpus to the target corpus

Upload the assignment until December 16th: https://forms.gle/1Q45azjwpKEV6jqz7


In [2]:
romanian_corpus = """A fost - ce n-a mai fost.
    A fost odată un lup, - Cumătrul-Nicola, şi-o vulpe, - Cumătra-Mara.
    Ăştia doi s-au întovărăşit, să vâneze împreună, iar vânatul, şi-au zis, l-or împărţi frăţeşte.
    Ieşind la vânat, dar, dau peste o oală cu miere.
    - Aha!, se repezi Cumătru-Nicola, - o pap!
    - Mai întâi şi mai întâi nu trebuie să spui: „o pap!“, ci „o păpăm“, i-a zis Cumătra-Mara. Apoi, nu te gândeşti, nici n-am dat bine de oală, şi gata: pe ea! Dacă vrei să facem casă laolaltă, află că n-o facem înghiţind orice agonisim...
    - Fie, Mara, cum ştii tu.
    - Da, cum ştiu eu! Să facem şi noi ca toată lumea. Acum, bunăoară, ascundem mierea şi din trei în trei zile mergem să mâncăm din ea.
    Făcură cum zise Cumătra-Mara cea şireată: au pitit mierea sub un jneapăn.
    Nu trece o zi şi iată că jupâneasa vulpe îi zice lupului:
    - Jupân Nicola, mă cheamă cineva să-i botez!
    - Dacă te cheamă, du-te!, zise lupul.
    Se duse vulpea şi - lip-lip! - înghiţi mierea de deasupra, că aci venise, ăsta-i era botezul.
    La întoarcere o întrebă lupul:
    - Ce nume i-ai dat pruncului, Cumătră-Mara?
    - Începutul! Cum era să-i zic altfel, primului băiat din casa omului?
    - Frumos nume! Să-ţi trăiască! - mai grăi lupul.
    A doua zi vulpea - iară:
    - Cumetre-Nicola, iar mă cheamă să botez.
    - Păi, du-te, dacă te cheamă!
    Plecă vulpea şi - lip-lip!, a înjumătăţit oala de miere.
    Când veni înapoi, lupul o întrebă din nou:
    - Ăstuia ce nume-i dăduşi, Cumătră-Mara?
    - Ăstuia-i zisei Jumăţică, fiindcă a fost fată de-astă dată.
    Mai trece o zi, şi vulpea:
    - Jupân Nicola, şi azi mă cheamă să botez!
    Lupul-lup, cum să priceapă el că aci nu-i lucru curat? „Du-te!“. Aşa i-a zis.
    Acuma vulpea a gătat toată mierea din oală!
    Întorcându-se ea de-acolo, Cumătru-Nicola întreabă iar, curios:
    - Ei, cum i-ai zis şi noului prunc?
    - Gătatu’ îi zisei - făcu vulpea, care ştia că pusese bine toată mierea.
    - Frumos nume, - să-ţi trăiască!, îi ură prostovanul de lup.
    Tocmai trecuseră cele trei zile, şi-acu trebuia să meargă să guste din mierea pitită. Dar, ce să mai guste? Pietre? Că mierea o hăpăise toată jupâneasa vulpe.
    - Cumetre-Nicola, Cumetre-Nicola, - ai păpat mierea singur, cum văd eu!, zise vulpea.
    - Ba, să mă ierţi, Cumătră-Mara, grăi lupul, dar mi se pare că tu ai făcut trebuşoara asta minunată!
    El, că tu, ea, că tu - s-au certat şi s-au despărţit.
    Vulpea o porni spre sat, să-nhaţe vreo găină. Pe drum se întâlneşte cu un popă. Popa se înturna de la biserică, spre casă, cu măgarul încărcat de prescuri. Ea îl văzuse mai de departe şi se-ntinse lângă cărare, prefăcându-se moartă. Popa, zărind-o, o ia de picior şi-o azvârle pe măgar, deasupra prescurilor. „E bună îşi zise omul, de-o blană pentru preoteasa mea!“
    Vulpea, în spate, dezleagă sacii şi aruncă prescurile una câte una, pe toate, în drum. Se dă apoi şi ea jos şi, din nou - una câte una - le culege pe toate şi, - pe-aci ţi-i drumul! Ajunge cu prescurile în pădure. Aci, iată-l şi pe Jupân-Nicola. Ăsta, uluit, întreabă:
    - De unde, cumătră, de unde, Mara, o aşa pleaşcă?
    - De unde să fie, Cumetre-Nicola? Păi, de la biserică!
    - Dar cum de ţi le-au dat? Cum făcuşi?
    - Cum făcui? Cum era să fac? Mă dusei lângă uşa bisericii şi-am început să-l toc pe popă: dă-mi, părinţele, o prescură! - dă-mi, părinţele, o prescură! Popa, ce putea face? Îmi dădu o desagă întreagă, în loc de o prescură!
    - Atunci, merg şi eu. Mi-o da şi mie?
    - Ba bine că nu! Cum să nu-ţi dea?!
    Lupul nu-şi pierde vremea şi, iată-l - la biserică. Îşi vâră capul pe uşă şi se porneşte pe un urlet - lupesc, ce mai!:
    - Dă-mi, părinte, o prescură! Dăăă-mi părinţele o prescură!
    - Huooo, lupe! Huooo, lupe! - începe toată lumea să ţipe - de n-a mai ştiut lupul cum să se facă nevăzut de pe acolo.
    Aşa, ajunse iar în pădure. Vulpea îi ieşi înainte şi-l întrebă cum s-a descurcat. Lupul îi povesti ce-a păţit şi cum erau să-l scarmene, dacă puneau mâna pe el.
    - Dar, spune-mi, cum ai cerut?, îl mai întrebă vulpea.
    - Păi, uite-aşa: Dăăă-mi, părinte, o prescură!
    - Ei, de-aia nu-ţi dădu, Cumetre-Nicola. Trebuie cerut cu o voce mai scăzută, nu aşa - urlând! Subţire, încetişor, aşa se cere: „Dă-mi, părinţele, o prescură“.
    Prostul de lup, care scăpase o dată cu viaţa, se lăsă dus de nas de vulpe, şi se mai duse o dată, să ceară acum... cu o voce mai scăzută. Numai că, nici nu apucă biet Cumătrul să-şi vâre bine capul pe uşă, şi lumea: Huooo, pe el! - cu câini, cu căţei, şi arde-l, şi arde-l cu parii, până îl omorâră!
    Şi, cu asta, vulpea a scăpat de neisprăvitul de lup."""

In [4]:
aromanian_corpus = """Eara ţe nu ş-eara.
    Eara un lup, - Masturlu-Nicola, şi ună vulpe, - Chira-Mara.
    Nîşi s-feaţiră soţ, se-avină deadun şi avinaticlu s-lu-mpartă frăţeşte.
    Tu işirea avinare - aflară ună oală cu ñeare.
    - Ha!, s-aurñi Masturlu-Nicola, - va u mîc!
    - Năinte ş-năinte nu prinde s-dzîţ: „va u mîc“, ma: „va u mîcăm“, îl’i faţe Chira-Mara. Ş-apoia, nu dzîţ tine, ninga nu o-aflăm - şi, ai! - s-u mîcăm? Macă vrei casă s-adrăm, casa nu se-adară cu gîlpuirea a tut ţe s-aflăm...
    - Cum ştii, Mară!
    - Ţi să ştiu? Cum fac toată lumea, ş-noi: s-o-ascundem ñearea şi dit trei-trei dzîle, s-nă duţem s-mîcăm cîte puţînă.
    Feaţiră cum gri Chira-Mara aţea şireata: o-ascumsiră ñearea tu un giuneapine.
    Nu tricu nă dzuă şi - ia-ţ-u cumbara vulpe, iu-l’i dzîţe a lupului:
    - Masture-Nicola, me-acl’imară s-pătedz!
    - Macă te-acl’imară, du-te!, dzîse luplu.
    Duse vulpea şi - glîp-glîp!, glîpui ñearea diprisupră, că aclo duse, aestu-l’i fu pătedzlu.
    Cîndu s-turnă, u întribă luplu:
    - Cum îl’i dzîsişi, Chiră-Mară?
    - A părnitulu! Cum altă soe s-l’i dzîţ, - că eara protlu ficior în casă.
    - Muşeată numă, s-ţă băneadză! - gri luplu.
    Dzua alantă, ia-ţ-u vulpea, pale:
    - Mastur-Nicola, me-acl’eamă s-pădetz, iară.
    - Em, du-te macă te-acl’eamă!
    Vulpea duse şi - glîp-glîp!, u-ngiumitică oala di ñeare.
    Tu turnată u-ntribă nîpoi luplu:
    - Ţi numă-l’i băgaşi, Chira-Mară?
    - Ţi numă s-l’i băgam? Giumitică îl’i dzîş, că-şi fu feată aestă.
    După nă dzuă, nîpoi, vulpea:
    - Mastur-Nicola, ş-adz me-acl’imară s-pătedz!
    Luplu-lup, iu si-l’i si ducă mintea lui că aoa nu cură lucru curat! „Du-te“, l’i-dzîse diznou.
    Cari duse vulpea, nu ţ-alăsă ñeare tu oală!
    Tu turnată, nîpoi u-ntribă Masturlu-Nicola:
    - E, ţi numă-l’i băgaşi ş-a noului vleat?
    - Biţită îl’i băgai numa!, dzîse vulpea - că ş-o-avea biţită tută ñarea.
    - Muşeată numă, - s-ţă bîneadză! - îl’i urlă zdanganlu di lup.
    Taman avea tricută treile dzîle, ş-aprindea să s-ducă s-mîcă di ñeare. Am, ţi s-mîcă, cheţre? Ñearea tută o-avea glîpuită mastorsa di vulpe.
    - Mastur-Nicola, Mastur-Nicola, - u mîcaşi ñearea fără di mine, cumu s-veade!, dzîse vulpea.
    - S-mi l’erţi, Chira-Mară, gri luplu - ma ñi si pare că tine u adraşi aestă muşuteaţă!
    Ba, că tine, ba, că mine - si-ncăceară şi si-mpărţîră.
    Vulpea lo calea cîtră-n hoară, s-anciupă vîră găl’ină. Aclo iu-şi imna, da di un preftu. Aestu s-turna, cu γumarlu încărcat di liturγii, di la bisearică. Nîsă-l vidzu, ma, di diparte, s-tease-n calea-l’i, ca psoahe, şi, preftul, cum u vidzu, cîndu tricu prin ningă vulpe, u lo di cicior ş-o-aruncă pri γumar. Bună-i, mindui omlu, tră guna ali priftease!
    Vulpea, pri dinăpoi, dizleagă saţl’i ş-arucă liturγiile ună cîte ună, tute îm-pade. Dipune apoia şi iar, ună cîte ună, le-adună di-mpade şi - l’ea-l’i ciulica, s-poţ! Agiumse cu nîse tu pădure. Aoa, na-l şi Masturlu-Nicola. Aestu-l’i dtîţe, ciudusit:
    - D-iu, cumbară, d-iu, Mară, cu pleaşca aestă?
    - D-iu s-hibă, Masture-Nicola? Em, di la bisearică!
    - Cum acşi, di ţî li deadiră? Cum feaţişi?
    - Cum s-fac? Cum vreai s-fac?! Mi duş ningă uşa di bisearică şi acăţai: dă-ñi, prefte, nă liturγie! - dă-ñi, prefte, nă liturγie! Şi preftul, tră ună - îñi deade nă tisagă!
    - S-mi duc ş-io, ñi-da ş-a-ñia?
    - Avdzî, s-da? Va zbor, aestă?!
    Nu amînă luplu şi na-l iu s-duţe la bisearică. Îşi bagă caplu tu uşă şi apărñeaşte s-aurlă ca vîr lup:
    - Dă-ñi, prefte, nă liturγie! Dăăă-ñi, prefte, nă liturγie! 
    - Duuua, lupe! Duuua, lupe! - acăţară lumea si zghilească di tute părţîle, - că nu vidzu luplu cum fudzi de-acloţi.
    Agiumse tu pădure. L’i-iase vulpea s-lu-aşteaptă şi s-lu-n-treabă, cum tricu? El aspuse ţi păţî ş-cum vrea-l’i frîngă chilunghea, s-lu băga tu mînă.
    - Am, nu ti-ntreb - cum dzîţeai?, gri vulpea.
    - Cum s-dzîc?: Dăăă-ñi, prefte, nă liturγie!
    - De-aestă nu-ţ deadiră, Mastur-Nicola. Va dzîţeare cu boaţea ma minută, nu tu groasa! Aclo va si stridz dip aγalea: „Dă-ñi, prefte, nă liturγie!“.
    Chirutlu di lup ascăpă nă oară nifrîmtu, şi s-lo după mintea ali vulpe, să s-ducă s-grească... cu boaţea minută. E, nu-apucă Masturlu si-şi bagă caplu tu uşe, şi: Duuuaaa! - pri nîs, cu cîñi, cu căţăl’i, şi-l cruiră, îl cruiră cu parlu, pînă-l vătămară!
    Ş-cu aestă, ascăpă vulpea di pondul di lup."""